artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．

このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．

JSON形式の概要は以下の通りである．

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．

KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．

ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

** 0. KVSの構築 **

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

*** leveldbパッケージを使った場合 ***

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test_db'

In [1]:
import leveldb

In [3]:
db = leveldb.LevelDB(fname_db)

In [25]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.Put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [27]:
Ndata = len(list(db.RangeIter(include_value=False)))

In [28]:
print("registered {} pairs".format(Ndata))

registered 921337 pairs


*** plyvelパッケージを使った場合***

https://plyvel.readthedocs.io/en/latest/user.html#getting-started

In [5]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test2_db'

In [6]:
import plyvel

In [8]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [9]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [10]:
len([key for key , value in db])

921337

In [11]:
db.close()

#### KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [30]:
db.RangeIter(k

In [26]:
%%bash
git commit -a -m "started 60"

[master 816c8d7] started 60
 2 files changed, 62 insertions(+), 8 deletions(-)
